In [1]:
# Import necessary libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import matplotlib.pyplot as plt

# --------------------------
# Half Adder Circuit
# --------------------------
# Inputs: a, b (qubits q0 and q1).
# Outputs:
#   - Sum: computed as a XOR b (stored in q1)
#   - Carry: computed as a AND b (stored in q2 using a Toffoli gate)
def half_adder_circuit():
    q = QuantumRegister(3, 'q')  # q0: a, q1: b, q2: ancilla for carry
    c = ClassicalRegister(2, 'c')  # c0: sum, c1: carry
    qc = QuantumCircuit(q, c)

    # Compute carry first: carry = a AND b
    qc.ccx(q[0], q[1], q[2])
    # Compute sum: sum = a XOR b (store result in q1)
    qc.cx(q[0], q[1])

    # Measure outputs
    qc.measure(q[1], c[0])  # Sum
    qc.measure(q[2], c[1])  # Carry
    return qc

# --------------------------
# Full Adder Circuit (Cuccaro style)
# --------------------------
# Inputs: a, b, carry_in (qubits q0, q1, q2)
# Outputs:
#   - Sum: stored in q1 (result of a ⊕ b ⊕ carry_in)
#   - Carry_out: stored in q0 (after applying the reversible "majority" then "unmajority" steps)
def full_adder_circuit():
    q = QuantumRegister(3, 'q')  # q0: a, q1: b, q2: carry in
    c = ClassicalRegister(2, 'c')  # c0: sum, c1: carry out
    qc = QuantumCircuit(q, c)

    # Cuccaro full adder implementation:
    qc.cx(q[0], q[1])
    qc.cx(q[0], q[2])
    qc.ccx(q[1], q[2], q[0])
    qc.cx(q[0], q[2])
    qc.cx(q[0], q[1])

    # Outputs: sum in q1, carry_out in q0.
    qc.measure(q[1], c[0])  # Sum
    qc.measure(q[0], c[1])  # Carry Out
    return qc

# --------------------------
# Half Subtractor Circuit
# --------------------------
# Inputs: a, b (qubits q0 and q1)
# Outputs:
#   - Difference: computed as a XOR b (stored in q1)
#   - Borrow: computed as (NOT a) AND b (stored in q2)
def half_subtractor_circuit():
    q = QuantumRegister(3, 'q')  # q0: a, q1: b, q2: ancilla for borrow
    c = ClassicalRegister(2, 'c')  # c0: difference, c1: borrow
    qc = QuantumCircuit(q, c)

    # Compute difference = a XOR b and store in q1
    qc.cx(q[0], q[1])

    # Compute borrow = (NOT a) AND b
    qc.x(q[0])
    qc.ccx(q[0], q[1], q[2])
    qc.x(q[0])

    qc.measure(q[1], c[0])  # Difference
    qc.measure(q[2], c[1])  # Borrow
    return qc

# --------------------------
# Full Subtractor Circuit
# --------------------------
# Implements a full subtractor using two half subtractors.
# Inputs: a, b, borrow_in (q0, q1, q2)
# We'll use additional ancillas:
#   q3: stores borrow from first half subtractor (borrow1 = (NOT a) AND b)
#   q4: stores borrow from second half subtractor (borrow2 = (NOT diff1) AND borrow_in)
#   q5: stores final borrow computed as OR(borrow1, borrow2)
# Output:
#   - Difference: (a ⊕ b) ⊕ borrow_in, stored in q1.
#   - Borrow_out: stored in q5.
def full_subtractor_circuit():
    q = QuantumRegister(6, 'q')
    c = ClassicalRegister(3, 'c')  # c0: difference, c1: borrow1 (for reference), c2: final borrow
    qc = QuantumCircuit(q, c)

    # First half subtractor on (a, b)
    # Compute diff1 = a XOR b and store in q1
    qc.cx(q[0], q[1])
    # Compute borrow1 = (NOT a) AND b into ancilla q3
    qc.x(q[0])
    qc.ccx(q[0], q[1], q[3])
    qc.x(q[0])

    # Second half subtractor on (diff1, borrow_in)
    # Compute final difference = diff1 XOR borrow_in stored in q1
    qc.cx(q[2], q[1])
    # Compute borrow2 = (NOT diff1) AND borrow_in into ancilla q4
    qc.x(q[1])
    qc.ccx(q[1], q[2], q[4])
    qc.x(q[1])

    # Compute final borrow as borrow1 OR borrow2.
    # Using De Morgan: borrow_out = NOT((NOT borrow1) AND (NOT borrow2))
    qc.x(q[3])
    qc.x(q[4])
    qc.ccx(q[3], q[4], q[5])
    qc.x(q[5])
    qc.x(q[3])
    qc.x(q[4])

    # Measure final outputs:
    # Difference in q1, borrow1 in q3 (optional), final borrow in q5.
    qc.measure(q[1], c[0])  # Difference
    qc.measure(q[3], c[1])  # borrow1 (for reference)
    qc.measure(q[5], c[2])  # Final Borrow
    return qc

# --------------------------
# Comparison Table: Circuit Depth and Size
# --------------------------
def print_comparison_table():
    circuits = {
        "Half Adder": half_adder_circuit(),
        "Full Adder": full_adder_circuit(),
        "Half Subtractor": half_subtractor_circuit(),
        "Full Subtractor": full_subtractor_circuit()
    }

    # Print header
    print("{:<18} {:<12} {:<12}".format("Circuit", "Depth", "Size"))
    print("-" * 42)

    # Compute metrics for each circuit
    for name, qc in circuits.items():
        depth = qc.depth()
        size = qc.size()
        print("{:<18} {:<12} {:<12}".format(name, depth, size))

    return circuits

# Draw circuits and run comparison
circuits = print_comparison_table()

# Display each circuit
for name, qc in circuits.items():
    print(f"\n{name} Circuit:")
    print(qc.draw())

Circuit            Depth        Size        
------------------------------------------
Half Adder         3            4           
Full Adder         6            7           
Half Subtractor    4            6           
Full Subtractor    10           17          

Half Adder Circuit:
                  
q_0: ──■────■─────
       │  ┌─┴─┐┌─┐
q_1: ──■──┤ X ├┤M├
     ┌─┴─┐└┬─┬┘└╥┘
q_2: ┤ X ├─┤M├──╫─
     └───┘ └╥┘  ║ 
c: 2/═══════╩═══╩═
            1   0 

Full Adder Circuit:
               ┌───┐             ┌─┐
q_0: ──■────■──┤ X ├──■────■─────┤M├
     ┌─┴─┐  │  └─┬─┘  │  ┌─┴─┐┌─┐└╥┘
q_1: ┤ X ├──┼────■────┼──┤ X ├┤M├─╫─
     └───┘┌─┴─┐  │  ┌─┴─┐└───┘└╥┘ ║ 
q_2: ─────┤ X ├──■──┤ X ├──────╫──╫─
          └───┘     └───┘      ║  ║ 
c: 2/══════════════════════════╩══╩═
                               0  1 

Half Subtractor Circuit:
          ┌───┐     ┌───┐   
q_0: ──■──┤ X ├──■──┤ X ├───
     ┌─┴─┐└───┘  │  └┬─┬┘   
q_1: ┤ X ├───────■───┤M├────
     └───┘     ┌─┴─┐ └╥┘ ┌─┐
q_2: ──────────